# প্রোডাকশনে AutoGen এজেন্ট: পর্যবেক্ষণ ও মূল্যায়ন

এই টিউটোরিয়ালে, আমরা শিখব কীভাবে **[Autogen এজেন্ট](https://github.com/microsoft/autogen)**-এর অভ্যন্তরীণ ধাপ (ট্রেস) পর্যবেক্ষণ করতে হয়** এবং **[Langfuse](https://langfuse.com) ব্যবহার করে এর কার্যকারিতা মূল্যায়ন করতে হয়।**

এই গাইডটি **অনলাইন** এবং **অফলাইন** মূল্যায়ন মেট্রিকস নিয়ে আলোচনা করে, যা দলগুলোকে দ্রুত এবং নির্ভরযোগ্যভাবে এজেন্ট প্রোডাকশনে নিয়ে যেতে সাহায্য করে।

**কেন AI এজেন্ট মূল্যায়ন গুরুত্বপূর্ণ:**
- যখন কোনো কাজ ব্যর্থ হয় বা অপ্রত্যাশিত ফলাফল দেয়, তখন সমস্যাগুলি ডিবাগ করা
- রিয়েল-টাইমে খরচ এবং কার্যকারিতা পর্যবেক্ষণ করা
- ধারাবাহিক প্রতিক্রিয়ার মাধ্যমে নির্ভরযোগ্যতা এবং নিরাপত্তা উন্নত করা


## ধাপ ১: পরিবেশ ভেরিয়েবল সেট করুন

আপনার Langfuse API কীসগুলো পেতে [Langfuse Cloud](https://cloud.langfuse.com/) এ সাইন আপ করুন অথবা [Langfuse নিজে হোস্ট করুন](https://langfuse.com/self-hosting)।

_**নোট:** যারা নিজে হোস্ট করছেন তারা [Terraform মডিউল](https://langfuse.com/self-hosting/azure) ব্যবহার করে Azure-এ Langfuse ডিপ্লয় করতে পারেন। বিকল্পভাবে, আপনি Kubernetes-এ Langfuse ডিপ্লয় করতে [Helm চার্ট](https://langfuse.com/self-hosting/kubernetes-helm) ব্যবহার করতে পারেন।_


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

পরিবেশ ভেরিয়েবল সেট করার পর, আমরা এখন Langfuse ক্লায়েন্ট আরম্ভ করতে পারি। `get_client()` পরিবেশ ভেরিয়েবলে প্রদত্ত শংসাপত্র ব্যবহার করে Langfuse ক্লায়েন্ট আরম্ভ করে।


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## ধাপ ২: OpenLit ইনস্ট্রুমেন্টেশন আরম্ভ করুন

এখন আমরা [OpenLit](https://github.com/openlit/openlit) ইনস্ট্রুমেন্টেশন আরম্ভ করি। OpenLit স্বয়ংক্রিয়ভাবে AutoGen অপারেশনগুলি ধারণ করে এবং OpenTelemetry (OTel) স্প্যানগুলি Langfuse-এ রপ্তানি করে।


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## ধাপ ৩: আপনার এজেন্ট চালান

এখন আমরা আমাদের যন্ত্রপাতি পরীক্ষা করার জন্য একটি বহু-পর্যায় এজেন্ট সেট আপ করব।


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### ট্রেস স্ট্রাকচার

Langfuse একটি **ট্রেস** রেকর্ড করে যা **স্প্যান** ধারণ করে, যা আপনার এজেন্টের লজিকের প্রতিটি ধাপকে উপস্থাপন করে। এখানে, ট্রেসটি এজেন্টের সামগ্রিক রান এবং সাব-স্প্যানগুলো অন্তর্ভুক্ত করে:
- মিল প্ল্যানার এজেন্ট
- নিউট্রিশনিস্ট এজেন্ট

আপনি এগুলো পরিদর্শন করতে পারেন, ঠিক কোথায় সময় ব্যয় হয়েছে, কতগুলো টোকেন ব্যবহার করা হয়েছে, ইত্যাদি দেখতে:

![Langfuse-এ ট্রেস ট্রি](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[ট্রেসের লিঙ্ক](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## অনলাইন মূল্যায়ন

অনলাইন মূল্যায়ন বলতে এজেন্টকে একটি লাইভ, বাস্তব পরিবেশে মূল্যায়ন করা বোঝায়, অর্থাৎ প্রকৃত উৎপাদন ব্যবহারের সময়। এর মধ্যে এজেন্টের কার্যকারিতা বাস্তব ব্যবহারকারীর ইন্টারঅ্যাকশনের উপর পর্যবেক্ষণ এবং ফলাফলগুলোর ধারাবাহিক বিশ্লেষণ অন্তর্ভুক্ত।

### উৎপাদনে ট্র্যাক করার সাধারণ মেট্রিক

1. **খরচ** — ইনস্ট্রুমেন্টেশন টোকেন ব্যবহারের তথ্য সংগ্রহ করে, যা আপনি প্রতি টোকেনের জন্য একটি নির্দিষ্ট মূল্য নির্ধারণ করে আনুমানিক খরচে রূপান্তর করতে পারেন।
2. **লেটেন্সি** — প্রতিটি ধাপ সম্পন্ন করতে বা পুরো রান শেষ করতে যে সময় লাগে তা পর্যবেক্ষণ করুন।
3. **ব্যবহারকারীর প্রতিক্রিয়া** — ব্যবহারকারীরা সরাসরি প্রতিক্রিয়া (লাইক/ডিসলাইক) প্রদান করতে পারে, যা এজেন্টকে উন্নত বা সংশোধন করতে সহায়তা করে।
4. **LLM-as-a-Judge** — আপনার এজেন্টের আউটপুট প্রায় রিয়েল-টাইমে মূল্যায়ন করতে একটি আলাদা LLM ব্যবহার করুন (যেমন, টক্সিসিটি বা সঠিকতা যাচাই করা)।

নিচে, আমরা এই মেট্রিকগুলোর উদাহরণ দেখিয়েছি।


#### ১. খরচ

নিচে একটি স্ক্রিনশট দেওয়া হয়েছে যেখানে `gpt-4o-mini` কলগুলোর ব্যবহার দেখানো হয়েছে। এটি ব্যয়বহুল ধাপগুলো চিহ্নিত করতে এবং আপনার এজেন্টকে অপ্টিমাইজ করতে সহায়ক।

![Costs](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[ট্রেসের লিঙ্ক](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### 2. লেটেন্সি

আমরা প্রতিটি ধাপ সম্পন্ন করতে কতক্ষণ সময় লেগেছে তাও দেখতে পারি। নিচের উদাহরণে, পুরো প্রক্রিয়াটি সম্পন্ন হতে প্রায় ৩ সেকেন্ড সময় লেগেছে, যা আপনি ধাপে ধাপে বিশ্লেষণ করতে পারেন। এটি আপনাকে বোতলজাত সমস্যা চিহ্নিত করতে এবং আপনার এজেন্টকে আরও কার্যকর করতে সাহায্য করে।

![Latency](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[ট্রেসের লিঙ্ক](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. ব্যবহারকারীর প্রতিক্রিয়া

যদি আপনার এজেন্টটি একটি ব্যবহারকারী ইন্টারফেসে এম্বেড করা থাকে, আপনি সরাসরি ব্যবহারকারীর প্রতিক্রিয়া রেকর্ড করতে পারেন (যেমন একটি চ্যাট ইন্টারফেসে থাম্বস-আপ/ডাউন)।


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

গুরুত্বপূর্ণ নিয়মাবলী:

1. ব্যবহারকারীর প্রতিক্রিয়া Langfuse-এ সংরক্ষণ করা হয়:

![ব্যবহারকারীর প্রতিক্রিয়া Langfuse-এ সংরক্ষণ করা হচ্ছে](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. স্বয়ংক্রিয় LLM-as-a-Judge স্কোরিং

LLM-as-a-Judge হলো আপনার এজেন্টের আউটপুট স্বয়ংক্রিয়ভাবে মূল্যায়নের আরেকটি পদ্ধতি। আপনি একটি আলাদা LLM কল সেটআপ করতে পারেন আউটপুটের সঠিকতা, বিষাক্ততা, শৈলী, বা যেকোনো অন্যান্য মানদণ্ড যাচাই করার জন্য যা আপনার গুরুত্বপূর্ণ।

**কাজের ধারা**:
1. আপনি একটি **মূল্যায়ন টেমপ্লেট** সংজ্ঞায়িত করেন, যেমন "পাঠ্যটি বিষাক্ত কিনা তা পরীক্ষা করুন।"
2. আপনি একটি মডেল সেট করেন যা বিচারক-মডেল হিসেবে ব্যবহৃত হয়; এই ক্ষেত্রে `gpt-4o-mini` যা Azure-এর মাধ্যমে প্রশ্ন করা হয়।
2. প্রতিবার আপনার এজেন্ট আউটপুট তৈরি করলে, আপনি সেই আউটপুটকে টেমপ্লেট সহ "বিচারক" LLM-এ পাঠান।
3. বিচারক LLM একটি রেটিং বা লেবেল দিয়ে সাড়া দেয় যা আপনি আপনার পর্যবেক্ষণ টুলে লগ করেন।

Langfuse থেকে উদাহরণ:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

আপনি দেখতে পাচ্ছেন যে এই উদাহরণের উত্তরের বিচার "অ-বিষাক্ত" হিসেবে করা হয়েছে।

![LLM-as-a-Judge মূল্যায়ন স্কোর](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### ৫. পর্যবেক্ষণযোগ্যতা মেট্রিক্সের সংক্ষিপ্ত বিবরণ

এই সমস্ত মেট্রিক্স একসাথে ড্যাশবোর্ডে প্রদর্শিত হতে পারে। এটি আপনাকে দ্রুত দেখতে সাহায্য করে যে আপনার এজেন্ট বিভিন্ন সেশনে কেমন পারফর্ম করছে এবং সময়ের সাথে সাথে গুণগত মেট্রিক্স ট্র্যাক করতে সহায়তা করে।

![পর্যবেক্ষণযোগ্যতা মেট্রিক্সের সংক্ষিপ্ত বিবরণ](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## অফলাইন মূল্যায়ন

অনলাইন মূল্যায়ন লাইভ প্রতিক্রিয়ার জন্য গুরুত্বপূর্ণ, তবে আপনাকে **অফলাইন মূল্যায়ন**ও করতে হবে—উন্নয়নের আগে বা চলাকালীন পদ্ধতিগত পরীক্ষা। এটি প্রোডাকশনে পরিবর্তন আনার আগে গুণমান এবং নির্ভরযোগ্যতা বজায় রাখতে সাহায্য করে।


### ডেটাসেট মূল্যায়ন

অফলাইন মূল্যায়নে, সাধারণত আপনি:
1. একটি বেঞ্চমার্ক ডেটাসেট রাখেন (যেখানে প্রম্পট এবং প্রত্যাশিত আউটপুট জোড়া থাকে)
2. আপনার এজেন্টকে সেই ডেটাসেটে চালান
3. আউটপুটগুলোকে প্রত্যাশিত ফলাফলের সাথে তুলনা করুন বা একটি অতিরিক্ত স্কোরিং পদ্ধতি ব্যবহার করুন

নিচে, আমরা এই পদ্ধতি [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) দিয়ে প্রদর্শন করছি, যেখানে প্রশ্ন এবং প্রত্যাশিত উত্তর রয়েছে।


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

পরবর্তী ধাপে, আমরা রান ট্র্যাক করার জন্য Langfuse-এ একটি ডেটাসেট এন্টিটি তৈরি করি। তারপর, আমরা ডেটাসেট থেকে প্রতিটি আইটেম সিস্টেমে যোগ করি।


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![ডেটাসেট আইটেমগুলি Langfuse-এ](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### ডেটাসেটে এজেন্ট চালানো

প্রথমে, আমরা একটি সহজ Autogen এজেন্ট তৈরি করি যা Azure OpenAI মডেল ব্যবহার করে প্রশ্নের উত্তর দেয়।


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

তারপর, আমরা একটি সহায়ক ফাংশন `my_agent()` সংজ্ঞায়িত করি।


In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

অবশেষে, আমরা প্রতিটি ডেটাসেট আইটেমের উপর লুপ করি, এজেন্ট চালাই, এবং ট্রেসটি ডেটাসেট আইটেমের সাথে সংযুক্ত করি। ইচ্ছা করলে আমরা একটি দ্রুত মূল্যায়ন স্কোরও সংযুক্ত করতে পারি।


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

আপনি এই প্রক্রিয়াটি বিভিন্ন এজেন্ট কনফিগারেশনের সাথে পুনরাবৃত্তি করতে পারেন যেমন:  
- মডেল (gpt-4o-mini, gpt-4.1, ইত্যাদি)  
- প্রম্পট  
- টুলস (সার্চ বনাম নো সার্চ)  
- এজেন্টের জটিলতা (মাল্টি এজেন্ট বনাম সিঙ্গেল এজেন্ট)  

তারপর এগুলোকে Langfuse-এ পাশাপাশি তুলনা করুন। এই উদাহরণে, আমি এজেন্টটি ২৫টি ডেটাসেট প্রশ্নে ৩ বার চালিয়েছি। প্রতিটি রানে, আমি একটি ভিন্ন Azure OpenAI মডেল ব্যবহার করেছি। আপনি দেখতে পাবেন যে বড় মডেল ব্যবহারের সময় সঠিকভাবে উত্তর দেওয়া প্রশ্নের সংখ্যা বৃদ্ধি পায় (যেমনটি প্রত্যাশিত)। `correct_answer` স্কোরটি একটি [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) দ্বারা তৈরি করা হয়েছে যা ডেটাসেটে দেওয়া নমুনা উত্তরের ভিত্তিতে প্রশ্নের সঠিকতা বিচার করার জন্য সেটআপ করা হয়েছে।  

![ডেটাসেট রান ওভারভিউ](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)  
![ডেটাসেট রান তুলনা](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)  



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় থাকা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
